In [13]:



import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from scipy.stats import norm






In [14]:
class diff_schemes:
    
    def __init__(self,dt,dx,r,x,t,u1,u2,initial_st):
        self.dt = dt
        self.dx = dx
        self.r = r # alpha*tol/h^2
        self.x = x
        self.t = t
        self.u1 = u1
        self.u2 = u2
        self.initial_st = initial_st
        
    def make_figure(self,matx,title_msg):
        fig = plt.figure()
        ax = fig.gca(projection = '3d')
        x, y = np.meshgrid(self.x, self.t)
        z = matx
        ax.plot_surface(x, y, z, cmap = cm.coolwarm, linewidth = 0, antialiased = False)
        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_zlabel('Z Label')
        plt.title(title_msg)
        plt.show()
        
    def init_condition(self):
        return self.initial_st
    
    def forward_diff_scheme(self):
        matx = np.zeros([len(self.t),len(self.x)])
        matx[0,:] = self.initial_st
        matx[:,0] = 0
        matx[:,-1] = 0
        for i in range(1, len(self.t)):
            for j in range(1,len(self.x)-1):
                """
                if self.x[j] == np.pi/3:
                    dd1 = 5
                elif self.x[j] == 2*np.pi/3:
                    dd2 = 5
                else:
                    dd1 = 0.0
                    dd2 = 0.0
                """
                dd1 = norm.pdf(self.x[j], np.pi/3, self.dx/(2**0.5))
                dd2 = norm.pdf(self.x[j], 2*np.pi/3, self.dx/(2**0.5))
                matx[i,j] = self.r*(matx[i-1,j-1]-2*matx[i-1,j]+matx[i-1,j+1]) + self.dt*(50*(np.exp(-4/(1+matx[i-1,j]))-np.exp(-4))) - self.dt*2*matx[i-1,j] + self.dt*2*((dd1/2)*self.u1+self.u2*(dd2/2)) + matx[i-1,j]
                if matx[i,j]<0:
                    matx[i,j] = 0
        return matx

In [15]:

x_step = 45
t_step = 200 

dt = 0.4/t_step
dx = np.pi/x_step
a = dt/dx**2
X_array = np.linspace(0,np.pi,(x_step+1))
X_array = np.array(X_array)
T_array = np.linspace(0,0.4,2*t_step+1)

i=0
xx = 7.5
initial_st = xx + 0*np.sin(X_array)
u1=0
u2=0
print(initial_st)
res = diff_schemes(dt,dx,a,X_array,T_array,u1,u2,initial_st)
ic = res.init_condition()
rfd = res.forward_diff_scheme()
print(rfd[-1])



[7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5
 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5
 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5 7.5]
[ 0.          1.17953799  2.33706685  3.45014642  4.50477251  5.49337447
  6.41254295  7.26147862  8.04099764  8.75289268  9.39951621  9.98350416
 10.50759019 10.97447963 11.38676334 11.74685883 12.05696993 12.31905947
 12.53483083 12.70571578 12.83286646 12.91715033 12.95914713 12.95914713
 12.91715033 12.83286646 12.70571578 12.53483083 12.31905947 12.05696993
 11.74685883 11.38676334 10.97447963 10.50759019  9.98350416  9.39951621
  8.75289268  8.04099764  7.26147862  6.41254295  5.49337447  4.50477251
  3.45014642  2.33706685  1.17953799  0.        ]


In [18]:
# In[3]:
import json

x_step = 45
t_step = 200 

dt = 0.4/t_step
dx = np.pi/x_step
a = dt/dx**2
X_array = np.linspace(0,np.pi,(x_step+1))
X_array = np.array(X_array)
T_array = np.linspace(0,0.4,2*t_step+1)
u1_arr = np.linspace(-200.0,0.0,21)
u2_arr = np.linspace(-200.0,0.0,21)
i=0
data = dict()
for u1 in u1_arr:
    for u2 in u2_arr:
        for xx in np.linspace(0.0,10.0,21):
            initial_st = xx
            #u1 = -200
            #u2 = -200
            res = diff_schemes(dt,dx,a,X_array,T_array,u1,u2,initial_st)
            ic = res.init_condition()
            rfd = res.forward_diff_scheme()
            #res.make_figure(rfd,'forward  a = %s'%a)
            #print(initial_st)
            #in_array = initial_st.append([u1,u2])
            in_array = np.hstack((initial_st,[u1,u2]))
            #print(in_array)
            title = str(xx) + "," + str(u1) + "," + str(u2)
            data[title] = rfd.tolist()

            """
            O = pd.DataFrame(rfd)
            I = pd.DataFrame(in_array)
            O.to_csv('output_f.csv', mode='a')
            I.to_csv('input_f.csv', mode='a')
            """
            print(i/len(u2_arr)/len(u1_arr)/21*100)
            i = i+1

with open("data3.json", 'w') as file_path:
    json.dump(data, file_path)



0.0
0.010797969981643452
0.021595939963286903
0.032393909944930355
0.043191879926573806
0.05398984990821725
0.06478781988986071
0.07558578987150415
0.08638375985314761
0.09718172983479105
0.1079796998164345
0.11877766979807797
0.12957563977972142
0.14037360976136484
0.1511715797430083
0.16196954972465177
0.17276751970629523
0.18356548968793868
0.1943634596695821
0.20516142965122558
0.215959399632869
0.22675736961451248
0.23755533959615593
0.2483533095777994
0.25915127955944284
0.2699492495410863
0.2807472195227297
0.2915451895043732
0.3023431594860166
0.31314112946766004
0.32393909944930355
0.334737069430947
0.34553503941259045
0.35633300939423385
0.36713097937587735
0.37792894935752086
0.3887269193391642
0.39952488932080765
0.41032285930245116
0.4211208292840946
0.431918799265738
0.44271676924738146
0.45351473922902497
0.46431270921066836
0.47511067919231187
0.4859086491739553
0.4967066191555988
0.5075045891372423
0.5183025591188857
0.5291005291005292
0.5398984990821726
0.550696469063